In [35]:
#Librarylist🤖
import glob, warnings, polars as pl, datetime, sqlite3, time, os, zipfile, xml.dom.minidom
from datetime import datetime as dt, time as t, timedelta, date
import pandas as pd, numpy as np, sqlalchemy as sa, xlsxwriter
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import win32com.client, xlwings as xw, psutil


# -----------------------------------------------------------------------------------------------#
# Source collection
# user_credential = os.path.join(os.environ['USERPROFILE'],r'Concentrix Corporation//CNXVN - WFM Team - Documents//')
user_credential = os.path.join(os.environ['USERPROFILE'],r'Concentrix Corporation//CNXVN - WFM Team - Documents//')

# [BKN]SEAT_MAP 📑
PRE_SEAT_Link = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//PRE_SEAT_DATA')
SEAT_MAP_Link = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//SEAT_MAP_DATA')
Seat_Declare = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//Seat_Declare.xlsx')
workbook_path = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//Map.xlsb')
workbook_link_path = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//SeatMap.xlsb')
seat_utilization_path = os.path.join(user_credential, 
                            r'DataBase//DataFrame//BKN//SEAT_MAP//SeatUtilization.xlsb')

# -----------------------------------------------------------------------------------------------#
# MyMiscellaneous

# [DB]TonyMiscellaneous
Link_DB = os.path.join(os.environ['USERPROFILE'], r'Desktop//Bcom_DB.db')
conn = create_engine(f"sqlite:///{Link_DB}")

In [36]:
#Get_Roster🗓️
SERVER_NAME = "PHMANVMDEV01V"
DATABASE = "wfm_vn_dev"
connection_string = f"mssql+pyodbc://{SERVER_NAME}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
engine = create_engine(connection_string)
sql_query = """ 

WITH
TL AS (SELECT [Employee_ID],[TED Name] from BCOM.Staff),
OT AS (SELECT [Emp ID],[Date],[OT] from BCOM.RegisteredOT),
ROSTER AS (
SELECT BCOM.ROSTER.[Attribute] as [Date],BCOM.ROSTER.[Emp ID],BCOM.Staff.[TED Name],
CASE WHEN OT.[OT]>0 AND BCOM.ROSTER.[Value]='OFF' then BCOM.ROSTER.[week_shift]
	 WHEN BCOM.ROSTER.[Value]='PEGA' THEN '1300-2200'
	 WHEN BCOM.ROSTER.[Value]='Training' THEN '0900-1800'
	 WHEN BCOM.ROSTER.[Value]='HAL' THEN BCOM.ROSTER.[week_shift]
ELSE BCOM.ROSTER.[Value] END AS [Shift],
TL.[TED Name] as [TL_Name],
ISNULL(OT.[OT],0) as [OT],
CASE WHEN BCOM.ROSTER.[Value] IN ('OFF', 'AL', 'CO', 'HO', 'UPL', 'VGH') THEN 'OFF'
	 WHEN BCOM.ROSTER.[Value] IN ('0000-0900', '0100-1000', '0200-1100', '0300-1200', '0400-1300', '0500-1400', '0600-1500', '0700-1600', '0800-1700', '0900-1800', '1000-1900', '1100-2000', '1200-2100', '1300-2200', '1400-2300') THEN 'DS'
	 WHEN BCOM.ROSTER.[Value] IN ('1500-0000', '1600-0100', '1700-0200', '1800-0300', '1900-0400', '2000-0500', '2100-0600', '2200-0700', '2300-0800') THEN 'NS'
	 ELSE Null END AS [Shift_type],
upper(left(DATENAME(weekday, BCOM.ROSTER.[Attribute]),3)) as [Week_day],
BCOM.Staff.[Booking Email]
FROM BCOM.ROSTER
LEFT JOIN BCOM.Staff ON BCOM.Staff.[Employee_ID]=BCOM.ROSTER.[Emp ID]
LEFT JOIN OT ON OT.[Emp ID]=BCOM.ROSTER.[Emp ID] and OT.[Date]=BCOM.ROSTER.[Attribute]
LEFT JOIN TL ON TL.[Employee_ID]=BCOM.ROSTER.[team_leader])
select * ,
CASE WHEN [Shift_type]='OFF' then 
DATEADD(second, 0,
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))
else 
DATEADD(second, -[OT]*3600,
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME))) end as [Start time],
CASE WHEN [Shift_type]='OFF' then 
DATEADD(second, 32400, 
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME))) 
ELSE DATEADD(second, 32400+[OT]*3600, 
DATEADD(SECOND,
DATEDIFF(SECOND,CAST('00:00:00' AS TIME), TIMEFROMPARTS(TRY_CAST(SUBSTRING([Shift], 1, 2) AS INT),TRY_CAST(SUBSTRING([Shift], 3, 2) AS INT), 0, 0, 0)),
CAST([Date] AS DATETIME)))
END as [End time],
CASE WHEN [TL_Name] in ('Anita Tran','John Luu','Yin Tran') THEN 0 ELSE 1 END AS [Check]

from ROSTER
where [Date]=DATEADD(DAY, +1,CAST(GETDATE() As Date)) and ([Shift_type]<>'OFF' or [Shift_type] is null or [OT]>0)
order by [Check],[Shift],[TL_Name] asc

"""    
 
Roster = pl.read_database(query=sql_query, connection=engine)
engine.dispose()
Roster = Roster.to_pandas()
Roster = Roster[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email']]
Roster[['Start time', 'End time']] = Roster[['Start time', 'End time']].apply(pd.to_datetime, format='mixed')

#Export
display(Roster)

,Date,Emp ID,TED Name,Shift,TL_Name,OT,Shift_type,Week_day,Start time,End time,Booking Email
0,2025-05-11,102269070,Ivy To,0400-1300,Yin Tran,0.0,DS,SUN,2025-05-11 04:00:00,2025-05-11 13:00:00,ivy.to@booking.com
1,2025-05-11,102514154,Kacei N,0400-1300,Yin Tran,0.0,DS,SUN,2025-05-11 04:00:00,2025-05-11 13:00:00,oanh.ngo@booking.com
2,2025-05-11,102231426,Thomas Vu,0400-1300,Yin Tran,0.0,DS,SUN,2025-05-11 04:00:00,2025-05-11 13:00:00,thomas.vu@booking.com
3,2025-05-11,102837790,Helen Tran,0400-1300,Yin Tran,0.0,DS,SUN,2025-05-11 04:00:00,2025-05-11 13:00:00,helen.tran@booking.com
4,2025-05-11,102238568,Alicee Nguyen,0800-1700,John Luu,4.0,OFF,SUN,2025-05-11 08:00:00,2025-05-11 17:00:00,alice.nguyen3@booking.com
...,...,...,...,...,...,...,...,...,...,...,...
287,2025-05-11,102217617,Jade Phan,2200-0700,Juan Nguyen,8.0,OFF,SUN,2025-05-11 22:00:00,2025-05-12 07:00:00,jade.phan@booking.com
288,2025-05-11,102517379,Tan Dao,2200-0700,Juan Nguyen,0.0,NS,SUN,2025-05-11 22:00:00,2025-05-12 07:00:00,tan.dao@booking.com
289,2025-05-11,102457380,Vogue Nguyen,2200-0700,Juan Nguyen,8.0,OFF,SUN,2025-05-11 22:00:00,2025-05-12 07:00:00,vogue.nguyen@booking.com
290,2025-05-11,102843531,Damian Bui,2200-0700,Juan Nguyen,2.0,NS,SUN,2025-05-11 20:00:00,2025-05-12 09:00:00,dong.bui@booking.com


In [37]:
#Get_SeatData🪑

#Import
today = pd.to_datetime("today")
dday = today.strftime('%Y-%m-%d')
filename="Previous_Seat_Data_" + dday
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")
seat_no = pd.read_excel(Seat_Declare,sheet_name="Seat No")
working_hrs=pd.read_excel(Seat_Declare,sheet_name="Working Hours")
trainee_seat=pd.read_excel(Seat_Declare,sheet_name="Trainee_Seat")
seat_no_update = pd.read_excel(PRE_SEAT_Link+"//"+filename+".xlsx",sheet_name="SEAT NO")

#Calculate Seat Hrs
seat_no['Date'] = pd.to_datetime(Roster['Date'].min(), format='mixed')
Roster_date = Roster[['Date', 'Week_day']].groupby(['Date', 'Week_day'], as_index=False).count()
working_hrs = pd.merge(working_hrs, Roster_date, on='Week_day', how='left')
working_hrs = working_hrs.dropna(subset=['Date'])
working_hrs['Date'] = pd.to_datetime(working_hrs['Date'])
#Create OpenHrs And CloseHrs
working_hrs['Open Hours'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' ' + working_hrs['Working hours (Open)'].astype(str), format='%Y-%m-%d %H:%M:%S')
working_hrs['Closed Hours'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' ' + working_hrs['Working hours (Closed)'].astype(str), format='%Y-%m-%d %H:%M:%S')
working_hrs['Check'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' 23:59:59', format='%Y-%m-%d %H:%M:%S')
#update OpenHrs
working_hrs['Closed Hours'] = working_hrs.apply(lambda x: x['Closed Hours'] + pd.Timedelta(days=1) if x['Closed Hours'] == x['Check'] else x['Closed Hours'], axis=1)
working_hrs = working_hrs.drop(columns=['Working hours (Open)', 'Working hours (Closed)', 'Floor', 'Building', 'Week_day', 'Check'])
seat_no=pd.merge(seat_no,working_hrs,left_on=['Code','Date'],right_on=['Code','Date'],how='left')
seat_no=pd.merge(seat_no,seat_no_update[['Seat No','Open Hours']],left_on=['Seat No'],right_on=['Seat No'],how='left')
#update OpenHrs
seat_no['Open Hours']=seat_no.apply(lambda x: x['Open Hours_y'] if x['Open Hours_y'] > x['Open Hours_x'] else x['Open Hours_x'], axis=1)
seat_no=seat_no.drop(columns=['Open Hours_x','Open Hours_y'])
#Reoder and Sort
seat_no['Check']=seat_no.apply(lambda x: 1 if x['Building'] =="QTSC1" else 0, axis=1)
seat_no=pd.merge(seat_no,trainee_seat,left_on=['Seat No'],right_on=['Seat No'],how='left')
seat_no['Open Hours']=seat_no.apply(lambda x: x['Datetime'] if x['Trainee seat'] =='Y' else x['Open Hours'], axis=1)
seat_no=seat_no.sort_values(by=['Check','Open Hours','Seat No'],ascending=False)
seat_no=seat_no[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]


#Export
display(seat_no)
seat_no.info()

,Seat No,Floor,Building,Code,Date,Open Hours,Closed Hours
42,Q014A044,4A,QTSC1,Q014A,2025-05-11,2025-05-11 08:00:00,2025-05-12 23:59:59
80,Q014A082,4A,QTSC1,Q014A,2025-05-11,2025-05-11 07:00:00,2025-05-12 23:59:59
40,Q014A042,4A,QTSC1,Q014A,2025-05-11,2025-05-11 07:00:00,2025-05-12 23:59:59
224,Q015A041,5A,QTSC1,Q015A,2025-05-11,2025-05-11 06:00:00,2025-05-12 23:59:59
215,Q015A032,5A,QTSC1,Q015A,2025-05-11,2025-05-11 06:00:00,2025-05-12 23:59:59
...,...,...,...,...,...,...,...
390,Q097F005,7F,QTSC9,Q097F,2025-05-11,2025-05-11 00:00:00,2025-05-12 23:59:59
389,Q097F004,7F,QTSC9,Q097F,2025-05-11,2025-05-11 00:00:00,2025-05-12 23:59:59
388,Q097F003,7F,QTSC9,Q097F,2025-05-11,2025-05-11 00:00:00,2025-05-12 23:59:59
387,Q097F002,7F,QTSC9,Q097F,2025-05-11,2025-05-11 00:00:00,2025-05-12 23:59:59


<class 'pandas.core.frame.DataFrame'>
Index: 481 entries, 42 to 386
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Seat No       481 non-null    object        
 1   Floor         481 non-null    object        
 2   Building      481 non-null    object        
 3   Code          481 non-null    object        
 4   Date          481 non-null    datetime64[ms]
 5   Open Hours    481 non-null    datetime64[ns]
 6   Closed Hours  481 non-null    datetime64[ns]
dtypes: datetime64[ms](1), datetime64[ns](2), object(4)
memory usage: 30.1+ KB


In [38]:
#Check Unseated Agents

#Loop1
ghe = seat_no
nhan_vien = Roster
#FIT_Seat
def tim_ghe_phu_hop(ca_bat_dau, ca_ket_thuc,teamlead):
    if teamlead in ('John Luu','Yin Tran','Anita Tran'):
        return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc) & (ghe['Code']=='Q014A')]['Seat No'].tolist()
    else:
        return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc)]['Seat No'].tolist()   
nhan_vien['ghe_phu_hop'] = nhan_vien.apply(lambda row: tim_ghe_phu_hop(row['Start time'], row['End time'],row['TL_Name']), axis=1)
#Seat_Mapping
ghe_da_chon = []
nhan_vien['ghe_duoc_chon'] = None
for i, row in nhan_vien.iterrows():
   for ghe_phu_hop in row['ghe_phu_hop']:
           if ghe_phu_hop not in ghe_da_chon:                   
                nhan_vien.loc[i, 'ghe_duoc_chon'] = ghe_phu_hop
                ghe_da_chon.append(ghe_phu_hop)
                break
ghe=ghe.merge(nhan_vien[['Emp ID','ghe_duoc_chon','End time']],left_on='Seat No', right_on='ghe_duoc_chon', how='left').fillna(0)
ghe['Open Hours']=ghe.apply(lambda x: x['End time'] if x['Emp ID'] !=0 else x['Open Hours'], axis=1)
ghe=ghe[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
nhan_vien=nhan_vien[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','ghe_duoc_chon']]
nhan_vien=nhan_vien.rename(columns={'ghe_duoc_chon':'Seat No'})

#loop2
for _ in range(4):  
    def tim_ghe_phu_hop(ca_bat_dau, ca_ket_thuc,teamlead):
        if teamlead in ('John Luu','Yin Tran','Anita Tran'):
            return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc) & (ghe['Code']=='Q014A')]['Seat No'].tolist()
        else:
            return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc)]['Seat No'].tolist()

    nhan_vien['ghe_phu_hop'] = nhan_vien.apply(lambda row: tim_ghe_phu_hop(row['Start time'], row['End time'],row['TL_Name']), axis=1)

    ghe_da_chon = []
    nhan_vien['ghe_duoc_chon'] = None
    for i, row in nhan_vien.iterrows():
       for ghe_phu_hop in row['ghe_phu_hop']:
           if ghe_phu_hop not in ghe_da_chon:
               if nhan_vien.loc[i,'Seat No'] == None:
                   nhan_vien.loc[i, 'ghe_duoc_chon'] = ghe_phu_hop
                   ghe_da_chon.append(ghe_phu_hop)           
                   break

    ghe=ghe.merge(nhan_vien[['Emp ID','ghe_duoc_chon','End time']],left_on='Seat No', right_on='ghe_duoc_chon', how='left')
    ghe['Emp ID']=ghe['Emp ID'].fillna(0)
    ghe['Open Hours']=ghe.apply(lambda x: x['End time'] if x['Emp ID'] !=0 else x['Open Hours'], axis=1)

    ghe=ghe[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
    nhan_vien['Seat No']=nhan_vien.apply(lambda x: x['ghe_duoc_chon'] if x['Seat No'] ==None else x['Seat No'], axis=1)
    nhan_vien=nhan_vien[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','Seat No']]

nhan_vien['Emp ID']=nhan_vien['Emp ID'].astype("string")
no_seat_list=nhan_vien.loc[(nhan_vien['Seat No'].isnull()==True)]
nhan_vien=nhan_vien.merge(ghe[['Seat No','Floor','Building']],left_on='Seat No', right_on='Seat No', how='left')

# Avail_Seat
occupied_seat = nhan_vien[['Seat No']].drop_duplicates()
occupied_seat.columns = ['Seat']
seat_available = pd.merge(ghe, occupied_seat, left_on='Seat No', right_on='Seat', how='left')
seat_available = seat_available.loc[seat_available['Seat'].isnull()]
pivot_seat=nhan_vien[['TED Name','Shift','TL_Name','Seat No','Floor','Building']]

#Convert to Pandas
nhan_vien = pl.from_pandas(nhan_vien)
ghe = pl.from_pandas(ghe)
no_seat_list = pl.from_pandas(no_seat_list)
seat_available = pl.from_pandas(seat_available)
pivot_seat = pl.from_pandas(pivot_seat)
pivot_seat = pivot_seat.pivot(index=['TL_Name', 'Shift'],on=['Building', 'Floor'],values='TED Name',aggregate_function='len')
pivot_seat = pivot_seat.sort(['TL_Name', 'Shift'])

# Export to CSV
def write_to_excel(filename, data_dict):
  with xlsxwriter.Workbook(filename) as workbook:
    for worksheet_name, data in data_dict.items():
      worksheet = workbook.add_worksheet(worksheet_name)
      data.write_excel(workbook=workbook, worksheet=worksheet_name, position="A1")
#Write to SEAT_MAP
date_1 = today + pd.Timedelta(days=1)
date_1 = date_1 .strftime('%Y-%m-%d')
seat_map_filename="SEAT_MAP_" + date_1
seat_data = {
    'OVERVIEW': pivot_seat,'SEAT MAP': nhan_vien,'AGENTS NO SEAT': no_seat_list,'SEAT AVAILABLE': seat_available,
}
write_to_excel(SEAT_MAP_Link+"//"+seat_map_filename+".xlsx", seat_data)
#Write to Previous_Seat_Data
os.chdir(PRE_SEAT_Link)
pre_seat_data_filename = "Previous_Seat_Data_" + date_1
pre_seat_data = ghe.write_excel(pre_seat_data_filename, worksheet = "SEAT NO")

#Display
display(nhan_vien)
display(no_seat_list)
display(seat_available)

C:\Users\thibaotram.nguyen\AppData\Local\Temp\ipykernel_15216\1219798825.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ghe['Emp ID']=ghe['Emp ID'].fillna(0)
C:\Users\thibaotram.nguyen\AppData\Local\Temp\ipykernel_15216\1219798825.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ghe['Emp ID']=ghe['Emp ID'].fillna(0)
C:\Users\thibaotram.nguyen\AppData\Local\Temp\ipykernel_15216\1219798825.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(c

Date,Emp ID,TED Name,Shift,TL_Name,OT,Shift_type,Week_day,Start time,End time,Booking Email,Seat No,Floor,Building
datetime[ms],str,str,str,str,f64,str,str,datetime[μs],datetime[μs],str,str,str,str
2025-05-11 00:00:00,"""102269070""","""Ivy To""","""0400-1300""","""Yin Tran""",0.0,"""DS""","""SUN""",2025-05-11 04:00:00,2025-05-11 13:00:00,"""ivy.to@booking.com""","""Q014A114""","""4A""","""QTSC1"""
2025-05-11 00:00:00,"""102514154""","""Kacei N""","""0400-1300""","""Yin Tran""",0.0,"""DS""","""SUN""",2025-05-11 04:00:00,2025-05-11 13:00:00,"""oanh.ngo@booking.com""","""Q014A112""","""4A""","""QTSC1"""
2025-05-11 00:00:00,"""102231426""","""Thomas Vu""","""0400-1300""","""Yin Tran""",0.0,"""DS""","""SUN""",2025-05-11 04:00:00,2025-05-11 13:00:00,"""thomas.vu@booking.com""","""Q014A108""","""4A""","""QTSC1"""
2025-05-11 00:00:00,"""102837790""","""Helen Tran""","""0400-1300""","""Yin Tran""",0.0,"""DS""","""SUN""",2025-05-11 04:00:00,2025-05-11 13:00:00,"""helen.tran@booking.com""","""Q014A107""","""4A""","""QTSC1"""
2025-05-11 00:00:00,"""102238568""","""Alicee Nguyen""","""0800-1700""","""John Luu""",4.0,"""OFF""","""SUN""",2025-05-11 08:00:00,2025-05-11 17:00:00,"""alice.nguyen3@booking.com""","""Q014A044""","""4A""","""QTSC1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-05-11 00:00:00,"""102217617""","""Jade Phan""","""2200-0700""","""Juan Nguyen""",8.0,"""OFF""","""SUN""",2025-05-11 22:00:00,2025-05-12 07:00:00,"""jade.phan@booking.com""","""Q014B055""","""4B""","""QTSC1"""
2025-05-11 00:00:00,"""102517379""","""Tan Dao""","""2200-0700""","""Juan Nguyen""",0.0,"""NS""","""SUN""",2025-05-11 22:00:00,2025-05-12 07:00:00,"""tan.dao@booking.com""","""Q014B025""","""4B""","""QTSC1"""
2025-05-11 00:00:00,"""102457380""","""Vogue Nguyen""","""2200-0700""","""Juan Nguyen""",8.0,"""OFF""","""SUN""",2025-05-11 22:00:00,2025-05-12 07:00:00,"""vogue.nguyen@booking.com""","""Q014B024""","""4B""","""QTSC1"""


Date,Emp ID,TED Name,Shift,TL_Name,OT,Shift_type,Week_day,Start time,End time,Booking Email,Seat No
datetime[ms],str,str,str,str,f64,str,str,datetime[μs],datetime[μs],str,str


Seat No,Floor,Building,Code,Date,Open Hours,Closed Hours,Seat
str,str,str,str,datetime[ms],datetime[ns],datetime[ns],null
"""Q014B021""","""4B""","""QTSC1""","""Q014B""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
"""Q014B020""","""4B""","""QTSC1""","""Q014B""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
"""Q014B019""","""4B""","""QTSC1""","""Q014B""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
"""Q014B017""","""4B""","""QTSC1""","""Q014B""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
"""Q014B016""","""4B""","""QTSC1""","""Q014B""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
…,…,…,…,…,…,…,…
"""Q097F005""","""7F""","""QTSC9""","""Q097F""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
"""Q097F004""","""7F""","""QTSC9""","""Q097F""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null
"""Q097F003""","""7F""","""QTSC9""","""Q097F""",2025-05-11 00:00:00,2025-05-11 00:00:00,2025-05-12 23:59:59,null


In [39]:
# Check if Unseated Agent is empty:
no_seat_list=no_seat_list.to_pandas()
if no_seat_list.empty:
    print("Done")
else:
    # Sort SEAT NO:
    dday = today.strftime('%Y-%m-%d')
    filename="Previous_Seat_Data_" + dday
    warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")
    seat_no = pd.read_excel(Seat_Declare,sheet_name="Seat No")
    working_hrs=pd.read_excel(Seat_Declare,sheet_name="Working Hours")
    seat_no_update = pd.read_excel(PRE_SEAT_Link+"//"+filename+".xlsx",sheet_name="SEAT NO")
    
    #Calculate Seat Hrs
    seat_no['Date'] = pd.to_datetime(Roster['Date'].min(), format='mixed')
    Roster_date = Roster[['Date', 'Week_day']].groupby(['Date', 'Week_day'], as_index=False).count()
    working_hrs = pd.merge(working_hrs, Roster_date, on='Week_day', how='left')
    working_hrs = working_hrs.dropna(subset=['Date'])
    working_hrs['Date'] = pd.to_datetime(working_hrs['Date'])
    #Create OpenHrs And CloseHrs
    working_hrs['Open Hours'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' ' + working_hrs['Working hours (Open)'].astype(str), format='%Y-%m-%d %H:%M:%S')
    working_hrs['Closed Hours'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' ' + working_hrs['Working hours (Closed)'].astype(str), format='%Y-%m-%d %H:%M:%S')
    working_hrs['Check'] = pd.to_datetime(working_hrs['Date'].astype(str) + ' 23:59:59', format='%Y-%m-%d %H:%M:%S')
    #update OpenHrs
    working_hrs['Closed Hours'] = working_hrs.apply(lambda x: x['Closed Hours'] + pd.Timedelta(days=1) if x['Closed Hours'] == x['Check'] else x['Closed Hours'], axis=1)
    working_hrs = working_hrs.drop(columns=['Working hours (Open)', 'Working hours (Closed)', 'Floor', 'Building', 'Week_day', 'Check'])
    seat_no=pd.merge(seat_no,working_hrs,left_on=['Code','Date'],right_on=['Code','Date'],how='left')
    seat_no=pd.merge(seat_no,seat_no_update[['Seat No','Open Hours']],left_on=['Seat No'],right_on=['Seat No'],how='left')
    #update OpenHrs
    seat_no['Open Hours']=seat_no.apply(lambda x: x['Open Hours_y'] if x['Open Hours_y'] > x['Open Hours_x'] else x['Open Hours_x'], axis=1)
    seat_no=seat_no.drop(columns=['Open Hours_x','Open Hours_y'])
    #Reoder and Sort
    seat_no=seat_no.sort_values(by=['Open Hours','Seat No'],ascending=False)
    seat_no=seat_no[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]


    #Mapping seat
    ghe = seat_no
    nhan_vien = Roster
    #FIT_Seat
    def tim_ghe_phu_hop(ca_bat_dau, ca_ket_thuc,teamlead):
        if teamlead in ('John Luu','Yin Tran','Anita Tran'):
            return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc) & (ghe['Code']=='Q014A')]['Seat No'].tolist()
        else:
            return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc)]['Seat No'].tolist()   
    nhan_vien['ghe_phu_hop'] = nhan_vien.apply(lambda row: tim_ghe_phu_hop(row['Start time'], row['End time'],row['TL_Name']), axis=1)
    #Seat_Mapping
    ghe_da_chon = []
    nhan_vien['ghe_duoc_chon'] = None
    for i, row in nhan_vien.iterrows():
       for ghe_phu_hop in row['ghe_phu_hop']:
               if ghe_phu_hop not in ghe_da_chon:                   
                    nhan_vien.loc[i, 'ghe_duoc_chon'] = ghe_phu_hop
                    ghe_da_chon.append(ghe_phu_hop)
                    break
    ghe=ghe.merge(nhan_vien[['Emp ID','ghe_duoc_chon','End time']],left_on='Seat No', right_on='ghe_duoc_chon', how='left').fillna(0)
    ghe['Open Hours']=ghe.apply(lambda x: x['End time'] if x['Emp ID'] !=0 else x['Open Hours'], axis=1)
    ghe=ghe[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
    nhan_vien=nhan_vien[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','ghe_duoc_chon']]
    nhan_vien=nhan_vien.rename(columns={'ghe_duoc_chon':'Seat No'})
    
    #loop2
    for _ in range(4):  
        def tim_ghe_phu_hop(ca_bat_dau, ca_ket_thuc,teamlead):
            if teamlead in ('John Luu','Yin Tran','Anita Tran'):
                return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc) & (ghe['Code']=='Q014A')]['Seat No'].tolist()
            else:
                return ghe[(ghe['Open Hours'] <= ca_bat_dau) & (ghe['Closed Hours'] >= ca_ket_thuc)]['Seat No'].tolist()
    
        nhan_vien['ghe_phu_hop'] = nhan_vien.apply(lambda row: tim_ghe_phu_hop(row['Start time'], row['End time'],row['TL_Name']), axis=1)
    
        ghe_da_chon = []
        nhan_vien['ghe_duoc_chon'] = None
        for i, row in nhan_vien.iterrows():
           for ghe_phu_hop in row['ghe_phu_hop']:
               if ghe_phu_hop not in ghe_da_chon:
                   if nhan_vien.loc[i,'Seat No'] == None:
                       nhan_vien.loc[i, 'ghe_duoc_chon'] = ghe_phu_hop
                       ghe_da_chon.append(ghe_phu_hop)           
                       break
    
        ghe=ghe.merge(nhan_vien[['Emp ID','ghe_duoc_chon','End time']],left_on='Seat No', right_on='ghe_duoc_chon', how='left')
        ghe['Emp ID']=ghe['Emp ID'].fillna(0)
        ghe['Open Hours']=ghe.apply(lambda x: x['End time'] if x['Emp ID'] !=0 else x['Open Hours'], axis=1)
    
        ghe=ghe[['Seat No','Floor','Building','Code','Date','Open Hours','Closed Hours']]
        nhan_vien['Seat No']=nhan_vien.apply(lambda x: x['ghe_duoc_chon'] if x['Seat No'] ==None else x['Seat No'], axis=1)
        nhan_vien=nhan_vien[['Date','Emp ID','TED Name','Shift','TL_Name','OT','Shift_type','Week_day','Start time','End time','Booking Email','Seat No']]
    
    nhan_vien['Emp ID']=nhan_vien['Emp ID'].astype("string")
    no_seat_list=nhan_vien.loc[(nhan_vien['Seat No'].isnull()==True)]
    nhan_vien=nhan_vien.merge(ghe[['Seat No','Floor','Building']],left_on='Seat No', right_on='Seat No', how='left')
    
    # Avail_Seat
    occupied_seat = nhan_vien[['Seat No']].drop_duplicates()
    occupied_seat.columns = ['Seat']
    seat_available = pd.merge(ghe, occupied_seat, left_on='Seat No', right_on='Seat', how='left')
    seat_available = seat_available.loc[seat_available['Seat'].isnull()]
    pivot_seat=nhan_vien[['TED Name','Shift','TL_Name','Seat No','Floor','Building']]
    
    #Convert to Pandas
    nhan_vien = pl.from_pandas(nhan_vien)
    ghe = pl.from_pandas(ghe)
    no_seat_list = pl.from_pandas(no_seat_list)
    seat_available = pl.from_pandas(seat_available)
    pivot_seat = pl.from_pandas(pivot_seat)
    pivot_seat = pivot_seat.pivot(index=['TL_Name', 'Shift'],on=['Building', 'Floor'],values='TED Name',aggregate_function='len')
    pivot_seat = pivot_seat.sort(['TL_Name', 'Shift'])
    
    # Export to CSV
    def write_to_excel(filename, data_dict):
      with xlsxwriter.Workbook(filename) as workbook:
        for worksheet_name, data in data_dict.items():
          worksheet = workbook.add_worksheet(worksheet_name)
          data.write_excel(workbook=workbook, worksheet=worksheet_name, position="A1")
    #Write to SEAT_MAP
    date_1 = today + pd.Timedelta(days=1)
    date_1 = date_1 .strftime('%Y-%m-%d')
    seat_map_filename="SEAT_MAP_" + date_1
    seat_data = {
        'OVERVIEW': pivot_seat,'SEAT MAP': nhan_vien,'AGENTS NO SEAT': no_seat_list,'SEAT AVAILABLE': seat_available,
    }
    write_to_excel(SEAT_MAP_Link+"//"+seat_map_filename+".xlsx", seat_data)
    #Write to Previous_Seat_Data
    os.chdir(PRE_SEAT_Link)
    pre_seat_data_filename = "Previous_Seat_Data_" + date_1
    pre_seat_data = ghe.write_excel(pre_seat_data_filename, worksheet = "SEAT NO")
    
    #Display
    display(nhan_vien)
    display(no_seat_list)
    display(seat_available)

Done


In [41]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = xw.Book(workbook_path)
wb.macro("Refresh")()
wb.save()

for proc in psutil.process_iter(['pid', 'name']):
    if proc.info['name'] == "EXCEL.EXE":  # Thay "EXCEL.EXE" bằng tên tiến trình Excel trên hệ thống của bạn
        proc.kill()

In [42]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = xw.Book(workbook_link_path)
wb.macro("Refresh")()
wb.save()

for proc in psutil.process_iter(['pid', 'name']):
    if proc.info['name'] == "EXCEL.EXE":  # Thay "EXCEL.EXE" bằng tên tiến trình Excel trên hệ thống của bạn
        proc.kill()

In [43]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

wb = xw.Book(seat_utilization_path)
wb.macro("Refresh_seat_utilization")()
wb.save()

for proc in psutil.process_iter(['pid', 'name']):
    if proc.info['name'] == "EXCEL.EXE":  # Thay "EXCEL.EXE" bằng tên tiến trình Excel trên hệ thống của bạn
        proc.kill()

In [44]:
%reset -f